In [ ]:
%load_ext autoreload
%autoreload 2

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loader" data-toc-modified-id="Data-loader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loader</a></span></li></ul></div>

# Подключение библиотек

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
from dataset import FiftyOneTorchDataset

# Download and load the validation split of COCO-2017
fo_dataset = foz.load_zoo_dataset("coco-2017", split="validation",
                               dataset_dir='/mnt/heap')

Found annotations at '/mnt/heap/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [22.2s elapsed, 0s remaining, 246.6 samples/s]      
Dataset 'coco-2017-validation' created


session = fo.launch_app(dataset)

In [3]:
fo_dataset.compute_metadata()

## Data set

In [4]:
import torch
from viewer import Viewer
from PIL import Image

In [5]:
viewer = Viewer()

In [6]:
from fiftyone import ViewField as F

person_list = ["person"]
person_view = fo_dataset.filter_labels("ground_truth",
        F("label").is_in(person_list)).match(F("ground_truth.detections").length() == 1)
print(len(person_view))

1045


## Data loader

In [7]:
import torchvision.transforms as T

img_size = (256, 256)
train_transforms = T.Compose([T.Resize(img_size),
                              T.ToTensor()])
test_transforms = T.Compose([T.Resize(img_size),
                             T.ToTensor()])

In [8]:
# split the dataset in train and test set
train_view = person_view.take(len(person_view) * 0.75, seed=51)
test_view = person_view.exclude([s.id for s in train_view])

In [9]:
torch_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=person_list)
torch_dataset_test = FiftyOneTorchDataset(test_view, test_transforms, 
        classes=person_list)

In [10]:
len(torch_dataset)

783

In [11]:
len(torch_dataset_test)

262

In [12]:
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=64)
    
data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=64, shuffle=False)

# Train loop

In [13]:
from itertools import product
from pytorch_helper import PyTorchHelper

In [14]:
loss_function_xy = torch.nn.MSELoss()
loss_function_bce = torch.nn.CrossEntropyLoss()

In [15]:
from extended_model import ExtendedModel

In [16]:
from pytorch_helper import PyTorchHelper

def find_hyperparameters(config):
    learning_rates = [1e-1]
    anneal_coeff = 0.2
    anneal_epochs = [10]
    regs = config['regs']
    optimizers = config['optimizers']

    batch_size = 64
    epoch_num = 20

    run_record = {} 
    
    helper = PyTorchHelper(8,  None)

    lenet_model = None
    val_loss = 8
    loss_history = None
    for lr, reg, anneal_epoch, optimizer in product(learning_rates, regs, anneal_epochs, optimizers):
        
        lenet_model = ExtendedModel(config['model'](), config['need_train'], config['model_name'])
        if not lenet_model.need_train:
            if lenet_model.load_model():
                return lenet_model
        
        optimizer = optimizer(lenet_model.torch_model.parameters(), lr=lr, weight_decay=reg)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=anneal_epoch, gamma=anneal_coeff)

        model, train_loss_history, val_loss_history, train_metric_history, val_metric_history = helper.train_model(lenet_model.torch_model, data_loader, data_loader_test, optimizer, epoch_num, scheduler)
        lenet_model.add_history(train_loss_history, val_loss_history, train_metric_history, val_metric_history)

    return lenet_model

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.models import resnet18

In [18]:
def net():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(in_features=512, out_features=5)
    return model

In [19]:
config = {
    'need_train': True,
    'regs': [0.001],
    'optimizers': [optim.Adam],
    'model': net,
    'model_name': 'best_lenet'    
}

In [20]:
from resource_monitor import ResourceMonitor
resourceMonitor = ResourceMonitor()

In [21]:
resourceMonitor.print_statistics('MB')

memory_allocated: 0.0
max_memory_allocated: 0.0
memory_reserved: 0.0
max_memory_reserved: 0.0


In [22]:
extended_model = find_hyperparameters(config)

Start train:
memory_allocated: 42.69677734375
max_memory_allocated: 42.69677734375
memory_reserved: 64.0
max_memory_reserved: 64.0
Epoch 0/19
----------
Step 0/13 Loss 0.8738172054290771
Step 1/13 Loss 23.410751342773438
Step 2/13 Loss 8.746126174926758
Step 3/13 Loss 6.634387493133545
Step 4/13 Loss 5.037987232208252
Step 5/13 Loss 5.254038333892822
Step 6/13 Loss 3.530411720275879
Step 7/13 Loss 3.571881055831909
Step 8/13 Loss 1.6356143951416016
Step 9/13 Loss 1.1877692937850952
Step 10/13 Loss 1.4743297100067139
Step 11/13 Loss 1.0375490188598633
Step 12/13 Loss 1.1649987697601318
Average loss train: 5.296638
Train map: 0.000000
Average loss test: 203161.765625
Test map: 0.000000
memory_allocated: 174.14208984375
max_memory_allocated: 3774.27001953125
memory_reserved: 4068.0
max_memory_reserved: 4068.0
Epoch 1/19
----------
Step 0/13 Loss 0.640064001083374
Step 1/13 Loss 0.6828130483627319
Step 2/13 Loss 0.4981065094470978
Step 3/13 Loss 0.31190362572669983
Step 4/13 Loss 0.3412937

In [ ]:
extended_model

In [23]:
from matplotlib import pyplot as plt

In [24]:
plt.plot(train_loss_history, label='train')
plt.plot(val_loss_history, label='val')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

NameError: name 'train_loss_history' is not defined

In [ ]:
resourceMonitor.print_statistics('MB')

In [ ]:
===============================================

In [ ]:
def predict(index):
    img, target = torch_dataset[index]
    pred = model(img.unsqueeze(0))[0]
    
    original_img = Image.open(target['img_path']).convert("RGB")
    viewer.print_prediction(original_img, target, pred)

In [ ]:
for i in range(160, 170):
    predict(i)

In [ ]:
===============================================

In [ ]:
===============================================

# Visualization

In [ ]:
import fiftyone as fo

def convert_torch_predictions(preds, det_id, s_id, w, h, classes):
    # Convert the outputs of the torch model into a FiftyOne Detections object
    dets = []
    #if preds[0] < 0.5:
     #   detections = fo.Detections(detections=dets)
      #  return detections, det_id
    
    # Parse prediction into FiftyOne Detection object
    x0,y0,x1,y1 = preds[1:]
    coco_obj = fouc.COCOObject(det_id, s_id, int(1), [x0, y0, x1-x0, y1-y0])
    det = coco_obj.to_detection((w,h), classes)
    dets.append(det)
    det_id += 1
        
    detections = fo.Detections(detections=dets)
        
    return detections, det_id

def add_detections(model, torch_dataset, view, field_name="predictions"):
    # Run inference on a dataset and add results to FiftyOne
    torch.set_num_threads(1)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    model.eval()
    model.to(device)
    image_paths = torch_dataset.img_paths
    classes = torch_dataset.classes
    det_id = 0
    
    with fo.ProgressBar() as pb:
        for img, targets in pb(torch_dataset):
            sample = view[targets["img_path"]]
            s_id = sample.id
            w = sample.metadata["width"]
            h = sample.metadata["height"]
            
            # Inference
            preds = model(img.unsqueeze(0).to(device))[0]
            
            detections, det_id = convert_torch_predictions(
                preds, 
                det_id, 
                s_id, 
                w, 
                h, 
                classes,
            )
            
            sample["predictions"] = detections
            sample.save()

In [ ]:
add_detections(model, torch_dataset_test, fo_dataset)

In [ ]:
session = fo.launch_app(fo_dataset)